In [1]:
# to set up new kernel for notebook:
# https://stackoverflow.com/questions/58119823/jupyter-notebooks-in-vscode-does-not-use-active-virtual-environment

# python3 -m ipykernel install --user --name=projectnam


# get dataframes of flood levels
from helper_methods import *
xml_root = get_xml_root('https://water.weather.gov/ahps2/hydrograph_to_xml.php?gage=rmdv2&output=xml')
observed_data = get_flood_data(xml_root, 'observed')
forecast_data = get_flood_data(xml_root, 'forecast')
bridge_forecast_data = bridge_to_fore(observed_data, forecast_data)


In [9]:
import dash
import dash_html_components as html
import plotly
from dash.dependencies import Input, Output
# MY IMPORTS
from helper_methods import *
from layout import *



##################################################################################
##################################_LAYOUT_########################################
##################################################################################
# Create App

app = dash.Dash(__name__)
app.layout = html.Div(
    html.Div([
        html.H4('James River Flood Forecast'),
        html.Div(id='live-update-text'),
        create_flood_graph(
            observed_df=get_observed_data(),
            forecast_df=get_forecast_data(),
        ),
        create_flood_interval(15)
    ])
)

if __name__ == '__main__':
    app.run_server(debug=True,port=6969)
    app.run_server(debug=True, dev_tools_silence_routes_logging = False)
    app.run_server(debug=True)


ModuleNotFoundError: No module named 'dash'

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(1, 1)

critical_zones = [{'zone': 2,'level': 9}, {'zone': 3,'level': 11}, {'zone': 4,'level': 12}, {'zone': 5,'level': 15}, {'zone': 6,'level': 22}] # excluding the record line, would be better as a line plot b/c it is a decimal number 
x_range = range(50)
x_values = []
y_values = []

critical_zones_index = 0
for zone in critical_zones:
    for x in x_range:
        for z in range(zone['zone']):
            x_values.append(x)
            y_values.append(zone['level'] + 1) # the plus on is because plotly's histogram colors the cell below the y-value
            lvl = zone['level'] + 1
        while lvl != critical_zones[critical_zones_index]['level']:
            lvl + 1
            for k in range(zone['zone']):
                x_values.append(x)
                y_values.append(lvl + 1)
    critical_zones_index += 1 

# xvalues = []
# for val in xvalues:
#     for i in range()
# print(xvalues)
fig.add_trace(go.Histogram2d(
    x = x_values,
    y = y_values,
    coloraxis = "coloraxis",
    xbins = {'start':1, 'size':1},
    ybins = {'start':1, 'size':1}, 
))

fig.show()


In [10]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=[1, 4], 
    y=[9, 9],
    fill=None,
    mode='lines',
    line_color='#ff0000',
    ))
fig.add_trace(go.Scatter(
    x=[1, 4],
    y=[12, 12],
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines', line_color='#ff0000'))
fig.add_trace(go.Scatter(
    x=[1, 4],
    y=[15, 15],
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines', line_color='#cc0000'))
fig.add_trace(go.Scatter(
    x=[1, 4],
    y=[22, 22],
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines', line_color='#990000'))

fig.show()

ModuleNotFoundError: No module named 'plotly'

In [5]:
from climacell_api.client import ClimacellApiClient
from pprint import pprint

key = "96Sx5iofKooIKqeBycfPBZfAmOTSnUa1"
client = ClimacellApiClient(key) 
# 4700 welby turn = (37.558331, -77.639555)
rt = client.realtime(lat=37.558, lon=-77.639, fields=[
    'temp',
    'feels_like',
    'humidity',
    'wind_speed',
    'wind_gust',
    'baro_pressure',
    'precipitation',
    'precipitation_type',
    # 'precipitation_probability',
    # 'precipitation_accumulation',
    'sunrise',
    'sunset',
    'visibility',
    'cloud_cover',
    'weather_code',
])
rt_data = rt.data()
rt_measurements = rt_data.measurements

def convert_to_farhrenheit(temp_c):
    temp_c /= 5
    temp_c *= 9
    temp_c += 32
    return temp_c

data = {
    'obs_time': rt_data.observation_time,
    'lat': rt_data.lat,
    'long': rt_data.lon,
    'temp_value': convert_to_farhrenheit(rt_measurements['temp'].value),
    'temp_units_units': 'F\N{DEGREE SIGN}',
    'feels_like_value': convert_to_farhrenheit(rt_measurements['feels_like'].value),
    'feels_like_units': 'F\N{DEGREE SIGN}',
    'humidity_value': rt_measurements['humidity'].value,
    'humidity_units': rt_measurements['humidity'].units,
    'wind_speed_value': rt_measurements['wind_speed'].value,
    'wind_speed_units': rt_measurements['wind_speed'].units,
    'wind_gust_value': rt_measurements['wind_gust'].value,
    'wind_gust_units': rt_measurements['wind_gust'].units,
    'baro_pressure_value': rt_measurements['baro_pressure'].value,
    'baro_pressure_units': rt_measurements['baro_pressure'].units,
    'precipitation_value': rt_measurements['precipitation'].value,
    'precipitation_units': rt_measurements['precipitation'].units,
    'precipitation_type_value': rt_measurements['precipitation_type'].value,
    'sunrise_value': rt_measurements['sunrise'].value,
    'sunset_value': rt_measurements['sunset'].value,
    'visibility_value': rt_measurements['visibility'].value,
    'visibility_units': rt_measurements['visibility'].units,
    'cloud_cover_value': rt_measurements['cloud_cover'].value,
    'cloud_cover_units': rt_measurements['cloud_cover'].units,
    'weather_code_value': rt_measurements['weather_code'].value,
}

pprint(data)


ModuleNotFoundError: No module named 'climacell_api'

In [17]:
import plotly.express as px
data = [ [1], [.8], [.6], [.4], [.2], [0]]
fig = px.imshow(data, color_continuous_scale=[
    'rgb(252, 227, 3)', 'rgb(128,0,38)', 
])
fig.show()

['rgb(255,255,204)', 'rgb(255,237,160)', 'rgb(254,217,118)', 'rgb(254,178,76)', 'rgb(253,141,60)', 'rgb(252,78,42)', 'rgb(227,26,28)', 'rgb(189,0,38)', 'rgb(128,0,38)']


In [18]:
import helper_methods as hm

hm.